In [1]:
import numpy as np
import pandas as pd
from numpy import log2 as log
eps = np.finfo(float).eps
import random
import math
from collections import Counter
from itertools import permutations

data = pd.read_csv('./dataset/train.csv')
data['Age'] = data['Age'].interpolate()
data.dropna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [2]:
data.loc[data['Age'] <= 16, 'Age'] = 0
data.loc[(data['Age'] > 16) & (data['Age'] < 50), 'Age'] = 1
data.loc[data['Age'] >= 50, 'Age'] = 2
data.loc[data['Fare'] <=15.64585, 'Fare'] = 0
data.loc[(data['Fare'] > 15.64585) & (data['Fare'] < 33), 'Fare'] = 1
data.loc[data['Fare'] >= 33, 'Fare'] = 2

data['Embarked'] = data['Embarked'].fillna(method = 'ffill')
data = data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'SibSp', 'Parch'], axis = 1)

In [3]:
def entropy(probs):  
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

In [4]:
def entropy_of_list(a_list):  
    cnt = Counter(x for x in a_list)   
    num_instances = len(a_list)
    probs = [x / num_instances for x in cnt.values()] 
    return entropy(probs)

In [5]:
def one_list(df_split, target_attribute_name):
    #df_split = df.groupby(i)
    nobs = len(data.index)
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    return(new_entropy)


In [7]:
def conditional_entropy(d, target_att):
    d1 = d.drop(target_att, axis = 1)
    print(d1.columns)
    l1 = list(permutations(d1.columns,len(d1.columns)))
    df = d.copy()
    #print(l1)
    for j in l1:
        l = []
        print("--------------------------------------------------")
        for i in j:
            l.append(i)
            print(l)
            con_entropy = one_list(df.groupby(l), 'Survived')
            print(con_entropy)
            
    
conditional_entropy(data, 'Survived')

Index(['Pclass', 'Sex', 'Age', 'Fare', 'Embarked'], dtype='object')
--------------------------------------------------
['Pclass']
0.8768768565796353
['Pclass', 'Sex']
0.6461338032372403
['Pclass', 'Sex', 'Age']
0.6021125185021272
['Pclass', 'Sex', 'Age', 'Fare']
0.5637158063543961
['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
0.5213543772825245
--------------------------------------------------
['Pclass']
0.8768768565796353
['Pclass', 'Sex']
0.6461338032372403
['Pclass', 'Sex', 'Age']
0.6021125185021272
['Pclass', 'Sex', 'Age', 'Embarked']
0.570521407472138
['Pclass', 'Sex', 'Age', 'Embarked', 'Fare']
0.5213543772825244
--------------------------------------------------
['Pclass']
0.8768768565796353
['Pclass', 'Sex']
0.6461338032372403
['Pclass', 'Sex', 'Fare']
0.6229909433163973
['Pclass', 'Sex', 'Fare', 'Age']
0.5637158063543961
['Pclass', 'Sex', 'Fare', 'Age', 'Embarked']
0.5213543772825244
--------------------------------------------------
['Pclass']
0.8768768565796353
['Pclass', 'S

0.5213543772825244
--------------------------------------------------
['Sex']
0.7430477952150327
['Sex', 'Age']
0.7251063085199967
['Sex', 'Age', 'Pclass']
0.6021125185021272
['Sex', 'Age', 'Pclass', 'Fare']
0.563715806354396
['Sex', 'Age', 'Pclass', 'Fare', 'Embarked']
0.5213543772825247
--------------------------------------------------
['Sex']
0.7430477952150327
['Sex', 'Age']
0.7251063085199967
['Sex', 'Age', 'Pclass']
0.6021125185021272
['Sex', 'Age', 'Pclass', 'Embarked']
0.570521407472138
['Sex', 'Age', 'Pclass', 'Embarked', 'Fare']
0.5213543772825245
--------------------------------------------------
['Sex']
0.7430477952150327
['Sex', 'Age']
0.7251063085199967
['Sex', 'Age', 'Fare']
0.6851940534472831
['Sex', 'Age', 'Fare', 'Pclass']
0.563715806354396
['Sex', 'Age', 'Fare', 'Pclass', 'Embarked']
0.5213543772825245
--------------------------------------------------
['Sex']
0.7430477952150327
['Sex', 'Age']
0.7251063085199967
['Sex', 'Age', 'Fare']
0.6851940534472831
['Sex', 'Age

0.9515948417266183
['Age', 'Fare']
0.8824680106892893
['Age', 'Fare', 'Pclass']
0.8118397413369672
['Age', 'Fare', 'Pclass', 'Embarked']
0.7751802806458941
['Age', 'Fare', 'Pclass', 'Embarked', 'Sex']
0.5213543772825244
--------------------------------------------------
['Age']
0.9515948417266183
['Age', 'Fare']
0.8824680106892893
['Age', 'Fare', 'Sex']
0.6851940534472832
['Age', 'Fare', 'Sex', 'Pclass']
0.5637158063543961
['Age', 'Fare', 'Sex', 'Pclass', 'Embarked']
0.5213543772825244
--------------------------------------------------
['Age']
0.9515948417266183
['Age', 'Fare']
0.8824680106892893
['Age', 'Fare', 'Sex']
0.6851940534472832
['Age', 'Fare', 'Sex', 'Embarked']
0.6409225549823624
['Age', 'Fare', 'Sex', 'Embarked', 'Pclass']
0.5213543772825244
--------------------------------------------------
['Age']
0.9515948417266183
['Age', 'Fare']
0.8824680106892893
['Age', 'Fare', 'Embarked']
0.8514166828428883
['Age', 'Fare', 'Embarked', 'Pclass']
0.7751802806458942
['Age', 'Fare', 'Em

0.5213543772825245
--------------------------------------------------
['Embarked']
0.9391245112397593
['Embarked', 'Pclass']
0.8593906228911666
['Embarked', 'Pclass', 'Sex']
0.627626854183961
['Embarked', 'Pclass', 'Sex', 'Fare']
0.5941031058655538
['Embarked', 'Pclass', 'Sex', 'Fare', 'Age']
0.5213543772825244
--------------------------------------------------
['Embarked']
0.9391245112397593
['Embarked', 'Pclass']
0.8593906228911666
['Embarked', 'Pclass', 'Age']
0.8211390073997061
['Embarked', 'Pclass', 'Age', 'Sex']
0.570521407472138
['Embarked', 'Pclass', 'Age', 'Sex', 'Fare']
0.5213543772825244
--------------------------------------------------
['Embarked']
0.9391245112397593
['Embarked', 'Pclass']
0.8593906228911666
['Embarked', 'Pclass', 'Age']
0.8211390073997061
['Embarked', 'Pclass', 'Age', 'Fare']
0.7751802806458941
['Embarked', 'Pclass', 'Age', 'Fare', 'Sex']
0.5213543772825245
--------------------------------------------------
['Embarked']
0.9391245112397593
['Embarked', 'Pc

In [10]:
def conditional_entropy2(d, target_att):
    d1 = d.drop(target_att, axis = 1)
    print(d1.columns)
    #l1 = list(permutations(d1.columns,len(d1.columns)))
    df = d.copy()
    #print(l1)
    for i in range(len(d1.columns)):
        l1 = list(permutations(d1.columns, i+1))
        for j in l1:
            l = []
            for k in j:
                l.append(i)
                print(l)
                con_entropy = one_list(df.groupby(l), 'Survived')
                print(con_entropy)
    
conditional_entropy2(data, 'Survived')

Index(['Pclass', 'Sex', 'Age', 'Fare', 'Embarked'], dtype='object')
[0]


KeyError: 0